In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Nov  9 17:29:50 2019

@author: Sungmin Hong
"""
#%%
import os
import re
import sys
import time
import json
from selenium import webdriver
from datetime import datetime
from datetime import date
from pymongo import MongoClient
#%%
# Global Variable
docNum = 0
#%%
# REPLACE With your LinkedIn Credentials
USERNAME = ""
PASSWORD = ""
#%%
def mongodb_init():
    client=MongoClient('mongodb://localhost')
    db=client.smartcareer
    return db
#%%
def mongodb_get_collection(db,item):
    col=db[item]
    return col
#%%
def mongodb_put_doc(doc):
    db=mongodb_init()
    col=mongodb_get_collection(db,'applicantprofile_analyst')

    try:
        global docNum
        re=col.insert_one(doc)
        ret=re.inserted_id    ##re에 id 번호 부여하는 것?
        docNum += 1
    except:
        ret=doc['ProfileID']
          
    return ret
#%%
def clean_item(item):
    item = item.replace('\n', ' ')
    item = item.strip()
    return item
#%%
# https://www.linkedin.com/search/results/people/?facetGeoRegion=%5B%22us%3A0%22%5D&keywords=Data%20Engineer&origin=FACETED_SEARCH
#검색하고자 하는 정보를 title를 넣어 기본 검색 URL을 만든다.
# 잡타이틀 정보에 불필요한 공백, 특수문자를 제거한다.
# 내가 가져올 url을 넘겨준다. scrape_url
def generate_scrape_url(scrape_url, config):

    title = config['Job Title']  ## config는 cfg.json전체 정보.

# collecting people in US only
    scrape_url += "/search/results/people/?facetGeoRegion=%5B%22us%3A0%22%5D&keywords="
    scrape_url += title
    scrape_url += "&origin=SWITCH_SEARCH_VERTICAL"


    valid_title_name = title.strip().replace(' ', '_')
    valid_title_name = re.sub(r'(?u)[^-\w.]', '', valid_title_name) #means 첫번째를 2번째로 바꾼다. valid_title_name에서

    return scrape_url
#%%
def scroll_down_page(browser, speed=8): 
    current_scroll_position, new_height = 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        browser.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = browser.execute_script("return document.body.scrollHeight")
        
#%%
def pscrape(config):
    start_time = time.time()
    # driver download: https://github.com/mozilla/geckodriver/releases
    # windows \, Linux and Max /
    driver = os.getcwd() + "/geckodriver.exe"
    base_url = "https://www.linkedin.com"
    sign_in_url = "https://www.linkedin.com/uas/login?fromSignIn=true"
    people_data = []
    page = 1

    USERNAME = config['User Name']
    PASSWORD = config['Password']
    col=config['Collection']
    
    # 주어진 json파일정보로 검색 url을 얻기
    people_search_url = generate_scrape_url(base_url, config)

    # 해당 URL 로 이동 (로그인 페이지 이동)
    print('\nSTATUS: Opening website')
    browser = webdriver.Firefox(executable_path=driver)  ##내가 Firefox에서 Chrome으로 바꿈
    browser.get(sign_in_url)
    time.sleep(1)

    # 해당 페이지에서 유저명과 패스워드를 입력.
    print('STATUS: Signing in')
    browser.find_element_by_id('username').send_keys(USERNAME)
    time.sleep(1)

    browser.find_element_by_id('password').send_keys(PASSWORD)
    time.sleep(1)

    # 로그인 버튼 클릭
    browser.find_element_by_class_name('login__form_action_container ').click()
    time.sleep(1)
 
    # 로그인 이후에 json에 주어진 jobtitle로 검색을 수행한다. 수행 후, 스크롤해주기
    print('STATUS: Searching for people\n')
    browser.get(people_search_url)
    time.sleep(2)
    scroll_down_page(browser, 4)

    # 사람 정보 위치(아마도)
    people = browser.find_elements_by_xpath('//div[@class="search-result__info pt3 pb4 ph0"]')

    
    if len(people) == 0:  # 만약 원하는 위치에 정보가 없으면
        print('STATUS: No people found. Press any key to exit scraper')
        print("Check docnum.txt for # of documents submitted!")
        # docnum.txt를 만들어서(또는 열어서) 정보를 써준다.
        today = datetime.now()
        f = open("docnum1.txt","w+")  ##docnum.txt 없으면 생기고, 쓰기 기능으로(+ 조사 필요)들어놓아야 하는지? 코드와 같은 Anaconda Folder 안에-
        f.write("Ran:\n")
        f.write(str(today))
        f.write("\n")
        f.write("\nNumber of documents submitted:\n")
        f.write(str(docNum))
        f.close()
        browser.quit()
        exit = input('')  # 어떤 역할인지? 우선은 skip(우선은 사용하는 곳이 없어보임.)
        sys.exit(0)  # 

    # 위치에 정보가 있으면
    
    #Scraping up to 6 pages per search.
    while True and page != 11:
        print('STATUS: Scraping Page ' + str(page))
        
        # 링크가져오기
        links = []
        for link in people:
            try:
                link = link.find_element_by_class_name("search-result__result-link").get_attribute("href")
                links.append(link)
            except:
                ()
                
        # 링크를 얻고 바로 그 주소로 이동.(획득한 링크만큼)
        for link in links:
            obj = {}  # 딕셔너리 형태로 빈 딕셔너리 생성.
            browser.get(link)
            time.sleep(2)

            # 20초동안 스크롤을 한다.
            scroll_down_page(browser, 20)
  
            # 아마도, 링크안에 ProfileID 관련 정보가 있다. 그래서 그 정보를 가져오기
            # obj : ProfileID 키 추가, 
            # case1. 값이 만약 people일 아닌 경우,
            obj['ProfileID'] = link.split('/')[len(link.split('/'))-2]
            if obj['ProfileID'] != "people":
                print("STATUS: Scraping Profile_ID: {}".format(obj['ProfileID']))

                # 무언가 클릭을 두번 한다. 
                try:
                    browser.find_element_by_xpath("//a[@class='lt-line-clamp__more']").click()
                except:
                    ()
                try:
                    browser.find_element_by_xpath("//button[@class='pv-profile-section__see-more-inline pv-profile-section__text-truncate-toggle link link-without-hover-state']").click()
                except:
                    ()

                # 어떤 요소의 Job Title를 text로 가져온다.
                try:
                    obj['Job Title'] = clean_item(browser.find_element_by_xpath("//h2[@class='mt1 t-18 t-black t-normal']").text)
                except:
                    obj['Job Title'] = ''

                # 어떤 요소의 Location을 가져온다. 
                try:
                    obj['Location'] = clean_item(browser.find_element_by_xpath("//li[@class='t-16 t-black t-normal inline-block']").text)
                except:
                    obj['Location'] = ''

                # Profile Summary를 가져오기 
                try:
                    obj['Profile Summary'] = clean_item(browser.find_element_by_class_name("pv-about__summary-text").text)
                except:
                    obj['Profile Summary'] = ''
                
                # 회사를 가져오기(근무했던 했던 회사가 아닐까?)
                try:
                    companies = browser.find_element_by_id("experience-section").find_elements_by_class_name("pv-profile-section__card-item-v2")
                except:
                    companies = []

                experience = []
                experience_obj = {}
                
                # 회사들(경력)에서 상세 정보가져오기
                for company in companies:
                    try:
                        experience_obj['Job Title'] = clean_item(company.find_element_by_tag_name('h3').text)
                    except:
                        experience_obj['Job Title'] = ''

                    try:
                        experience_obj['Company'] = clean_item(company.find_element_by_class_name('pv-entity__secondary-title').text)
                    except:
                        experience_obj['Company'] = ''

                    try:
                        experience_obj['Period'] = clean_item(company.find_element_by_class_name('pv-entity__date-range').text).replace('Dates Employed ', '')
                    except:
                        experience_obj['Period'] = ''

                    try:
                        experience_obj['Years'] = clean_item(company.find_element_by_class_name("pv-entity__bullet-item-v2").text)
                    except:
                        experience_obj['Years'] = ''

                    try:
                        experience_obj['Location'] = clean_item(company.find_element_by_class_name('pv-entity__location').text).replace('Location ', '')
                    except:
                        experience_obj['Location'] = ''

                    try:
                        experience_obj['Description'] = clean_item(company.find_element_by_class_name('pv-entity__description').text)
                    except:
                        experience_obj['Description'] = ''

                    experience.append(experience_obj)
                    experience_obj = {}

                obj['Experience'] = experience

                # 어떤 교육(대학 또는 기관)을 받았는가?
                try:
                    institutes = browser.find_element_by_id("education-section").find_elements_by_class_name("pv-entity__summary-info")
                except:
                    institutes = []

                education = []
                education_obj = {}

                # 어떤 학교?(school) 전공일 것 같음. 아마. 
                for institute in institutes:
                    try:
                        education_obj['School'] = clean_item(institute.find_element_by_xpath("//h3[@class='pv-entity__school-name t-16 t-black t-bold']").text)
                    except:
                        education_obj['School'] = ''

                    try:
                        degree_name = clean_item(institute.find_element_by_class_name('pv-entity__degree-name').text).replace('Degree Name ', '')
                    except:
                        degree_name = ''

                    try:
                        field_of_study = clean_item(institute.find_element_by_class_name('pv-entity__fos').text).replace('Field Of Study ', '')
                    except:
                        field_of_study = ''

                    try:
                        grade = clean_item(institute.find_element_by_class_name('pv-entity__grade').text).replace('Grade ', '')
                    except:
                        grade = ''

                    education_obj['Degree'] = degree_name+' '+field_of_study+' '+grade

                    try:
                        education_obj['Date Attended'] = clean_item(institute.find_element_by_class_name('pv-entity__dates').text).replace('Dates attended or expected graduation ', '')
                    except:
                        education_obj['Date Attended'] = ''

                    education.append(education_obj)
                    education_obj = {}

                obj['Education'] = education
            
                try:
                    browser.find_element_by_xpath("//button[@class='pv-profile-section__card-action-bar pv-skills-section__additional-skills artdeco-container-card-action-bar artdeco-button artdeco-button--tertiary artdeco-button--3 artdeco-button--fluid']").click()
                except:
                    print("No Skills")

                try:
                    skill_sets = browser.find_element_by_class_name("pv-skill-categories-section__top-skills").find_elements_by_class_name("pv-skill-category-entity__skill-wrapper")
                except:
                    skill_sets = []

                skills = []
                skills_obj = {}

                # 당신이 가진 기술은 (아마 확인 필요. Python, R, Spark, ...)
                for skill_set in skill_sets:
                    try:
                        skills_obj['Skills'] = clean_item(skill_set.find_element_by_class_name('pv-skill-category-entity__name').text)
                    except:
                        skills_obj['Skills'] = ''

                    skills.append(skills_obj)
                    skills_obj = {}

                obj['Skills & Endorsements'] = skills

                try:
                    all_skillsets = browser.find_elements_by_xpath("//div[@class='pv-skill-category-list pv-profile-section__section-info mb6 ember-view']")
                except:
                    all_skillsets = []

                # 스킬들에 정보를 하나씩 뽑아내기
                for one in all_skillsets:
                    general_skills = []
                    try:
                        g_skills = one.find_elements_by_class_name("pv-skill-category-entity")
                        for g_skill in g_skills:
                            try:
                                skills_obj['Skills'] = clean_item(
                                    g_skill.find_element_by_class_name('pv-skill-category-entity__name').text)
                            except:
                                skills_obj['Skills'] = ''

                            general_skills.append(skills_obj)
                            skills_obj = {}
                    except:
                        general_skills = []

                    try:
                        category_name = clean_item(one.find_element_by_tag_name('h3').text)
                    except:
                        category_name = "Skills"

                    obj[category_name] = general_skills

                try:
                    dateCaptured = date.today()
                    obj['Date Captured'] = str(dateCaptured)
                except:
                    obj['Date Captured'] = ''

                # 이제 모아진 정보 (obj에 모였음) 이를 mongodb에 넣는다.
                people_data.append(obj)
            
                doc_id=mongodb_put_doc(obj)   # 넣기
                print('post id: ', doc_id)
            else:
                print("STATUS: Skipping Profile_ID: {}".format(obj['ProfileID']))

        page += 1
        
        # 다음 페이지 넘기기
        next_page = people_search_url + '&page=' + str(page)
        browser.get(next_page)
        time.sleep(2)
        scroll_down_page(browser, 4)


        people = browser.find_elements_by_xpath('//div[@class="search-result__info pt3 pb4 ph0"]')
        if len(people) == 0:
            break

        # 앞의 while문에 의해서 아마 6페이지를 가져오는 소스 코드임.
    browser.quit()
#%%
import os

if __name__ == '__main__':
    
    # Reads in the config file so input is automatic.
    with open("cfg.json") as json_cfg:
        d = json.load(json_cfg)
    
    jobTitle = d['Job Title']

    jobList = []
    tempStr = ""

    pscrape(d)
    # while len(configArray) > 0:
    #     try:
    #         int(configArray[0])
    #         break
    #     except:
    #         jobList.append(configArray[0])
    #         del configArray[0]

    # # Searches multiple jobs in a row.
    # for x in range(len(jobList)):
    #     print("Now scraping:", jobList[0])
    #     pscrape(jobList[0], configArray)
    #     time.sleep(5)
    #     del jobList[0]

    print("Daily automation has been completed for: get_people.py")
    print("Check docnum.txt for # of documents submitted!")
    today = datetime.now()
    f = open("docnum.txt","w+")
    f.write("Ran:\n")
    f.write(str(today))
    f.write("\n")
    f.write("\nNumber of documents submitted:\n")
    f.write(str(docNum))
    f.close()

    sys.exit(0)


STATUS: Opening website
STATUS: Signing in
STATUS: Searching for people

STATUS: Scraping Page 1
STATUS: Scraping Profile_ID: emmabobaekong
post id:  5dc67f352d165786280a5e89
STATUS: Scraping Profile_ID: jinwoo-kang-097b0977
post id:  5dc67f512d165786280a5e8b
STATUS: Scraping Profile_ID: suzannesoojungbae
post id:  5dc67f692d165786280a5e8d
STATUS: Scraping Profile_ID: hayliekim
post id:  5dc67f832d165786280a5e8f
STATUS: Scraping Profile_ID: angeline-peng
post id:  5dc67f9e2d165786280a5e91
STATUS: Scraping Profile_ID: ieum-han-365533b4
No Skills
post id:  5dc67fc12d165786280a5e93
STATUS: Scraping Profile_ID: jihye-park-5283a6135
post id:  5dc67fda2d165786280a5e95
STATUS: Scraping Profile_ID: kellyyeonjikim1027
No Skills
post id:  5dc67fec2d165786280a5e97
STATUS: Scraping Profile_ID: aneceahn
post id:  5dc67fff2d165786280a5e99
STATUS: Scraping Profile_ID: shelbykim
post id:  5dc680182d165786280a5e9b
STATUS: Scraping Page 2
STATUS: Scraping Profile_ID: reywon
post id:  5dc680772d16578628

Daily automation has been completed for: get_people.py
Check docnum.txt for # of documents submitted!


SystemExit: 0

C:\Python\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
